<a href="https://colab.research.google.com/github/ReemaAlharbi01/Recomandation-system/blob/main/Rec_task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Reading the dataset**

In [ ]:
!pip install pyxlsb
from sklearn.metrics.pairwise import cosine_distances
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display
import numpy as np
from google.colab import drive
import matplotlib.pyplot as plt
import pandas as pd
import re

drive.mount('/content/drive')
df = pd.read_excel('/content/drive/My Drive/Virtual/stcRecomandations_T3.xlsb')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Data cleaning**

In [ ]:
df.shape

(1048575, 6)

In [ ]:
df.head(5)

,Unnamed: 0,user_id_maped,program_name,rating,date_,program_genre
0,0,26138,100 treets,1,2017-05-27,Drama
1,1,7946,Moana,1,2017-05-21,Animation
2,2,7418,The Mermaid Princess,1,2017-08-10,Animation
3,3,19307,The Mermaid Princess,2,2017-07-26,Animation
4,4,15860,Churchill,2,2017-07-07,Biography


`Remove any space or symbols`

In [ ]:
def clean_title(title):
  title_2=re.sub("[^a-zA-Z0-9 ]","", title)
  cleaned_title=title_2.lower()
  return cleaned_title

In [ ]:
#new column include the cleaned program names
df['cleaned_title']=df['program_name'].apply(clean_title)

In [ ]:
df.head()

,Unnamed: 0,user_id_maped,program_name,rating,date_,program_genre,cleaned_title
0,0,26138,100 treets,1,2017-05-27,Drama,100 treets
1,1,7946,Moana,1,2017-05-21,Animation,moana
2,2,7418,The Mermaid Princess,1,2017-08-10,Animation,the mermaid princess
3,3,19307,The Mermaid Princess,2,2017-07-26,Animation,the mermaid princess
4,4,15860,Churchill,2,2017-07-07,Biography,churchill


`Add new column for program_id`

In [ ]:
program_dict = {} #store the programs name
program_id_counter = 1 #for the id

for index, row in df.iterrows():
    program_name = row['program_name']

    if program_name in program_dict:
        program_id = program_dict[program_name]
    else:
        program_id = program_id_counter
        program_dict[program_name] = program_id
        program_id_counter += 1

    df.at[index, 'program_id'] = program_id



In [ ]:
df.head()

,Unnamed: 0,user_id_maped,program_name,rating,date_,program_genre,cleaned_title,program_id
0,0,26138,100 treets,1,2017-05-27,Drama,100 treets,1.0
1,1,7946,Moana,1,2017-05-21,Animation,moana,2.0
2,2,7418,The Mermaid Princess,1,2017-08-10,Animation,the mermaid princess,3.0
3,3,19307,The Mermaid Princess,2,2017-07-26,Animation,the mermaid princess,3.0
4,4,15860,Churchill,2,2017-07-07,Biography,churchill,4.0


**Preparing the data**

In [ ]:
# it converts the feature to a number.
vectorizer=TfidfVectorizer(ngram_range=(1,3))
tfidf= vectorizer.fit_transform(df['cleaned_title'])

In [ ]:
#new data with the needed feature only
ratings_df = pd.DataFrame(df[['user_id_maped','program_id','rating']])

In [ ]:
ratings_df.head()

,user_id_maped,program_id,rating
0,26138,1.0,1
1,7946,2.0,1
2,7418,3.0,1
3,19307,3.0,2
4,15860,4.0,2


**create similarty table**

In [ ]:
program_id = 2 #for example
#def find the similler programs:
programs = df[df["program_id"] == program_id]


In [ ]:
programs

,Unnamed: 0,user_id_maped,program_name,rating,date_,program_genre,cleaned_title,program_id
1,1,7946,Moana,1,2017-05-21,Animation,moana,2.0
18,18,26546,Moana,1,2017-06-25,Animation,moana,2.0
19,19,20306,Moana,1,2017-06-23,Animation,moana,2.0
48,48,28687,Moana,2,2017-05-30,Animation,moana,2.0
191,191,16739,Moana,1,2017-05-27,Animation,moana,2.0
...,...,...,...,...,...,...,...,...
1047660,1047660,22833,Moana,3,2017-06-03,Animation,moana,2.0
1047786,1047786,32606,Moana,2,2017-05-10,Animation,moana,2.0
1047833,1047833,23131,Moana,2,2017-05-13,Animation,moana,2.0
1048463,1048463,16603,Moana,1,2017-06-23,Animation,moana,2.0


In [ ]:
#dfined all the programs that have the same searched program ID and are rated good.
#return the user id that prefair the searched program
similar_users = ratings_df[(ratings_df["program_id"] == program_id) & (ratings_df["rating"] >= 4)]["user_id_maped"].unique()

In [ ]:
similar_users

array([23732, 32443, 27549, 32439, 14274, 23425, 28506,  1630,  2406,
       27198, 17206, 18800, 12213, 14987,  2015, 26199, 15719, 12251,
        1321, 26769, 20249,  7969, 15637,  8030, 11660, 16327, 20716,
       16558, 32948, 25976, 30851, 18777, 24306, 16680, 17164, 18264,
       18617, 18127,  2120, 33871,  2219, 17700, 33705, 33745, 12041,
       26136,  9664, 24836,  4667, 18083, 15067,  5275, 20737, 15799,
       29829, 26929, 28418,  6009, 29360, 12459, 19326,  1670, 33171,
       10449, 29421,  9317,  4425, 19044, 22325, 13862, 23261, 27892,
       20226,  8228,  4074,  7294,  5094,  8682, 26704,  7519,  6274,
       20807, 33579,  3988, 33135, 12413, 29655,  7134, 15839, 18813,
         671, 28193, 27299,  9761, 25917,  6672, 27032, 14304, 24428,
       27491, 10926, 11132,  5540, 30500, 12171, 11477, 20548, 16901,
       22920,  4049,  6201, 27331,  9765,  3256,  1847, 32490,   343,
        9498, 23299,  5124, 14241, 17610, 24618,  6497, 23118,  9780,
       15965, 20014,

In [ ]:
# Pring all the users that like the same reseached program and Pring all their preferred programs
similar_user_programs= ratings_df[(ratings_df["user_id_maped"].isin(similar_users)) & (ratings_df["rating"] >= 4)]["program_id"]


In [ ]:
similar_user_programs

68           51.0
332         182.0
545         260.0
903         394.0
1064         25.0
            ...  
1048341    4077.0
1048465     136.0
1048469     450.0
1048474    1335.0
1048570      29.0
Name: program_id, Length: 21059, dtype: float64

In [ ]:
similar_user_programs = similar_user_programs.value_counts() / len(similar_users)

similar_user_programs = similar_user_programs[similar_user_programs > .10]


In [ ]:
similar_user_programs

2.0       2.850174
53.0      1.585366
24.0      1.104530
709.0     0.933798
603.0     0.459930
            ...   
733.0     0.101045
1642.0    0.101045
3560.0    0.101045
3066.0    0.101045
530.0     0.101045
Name: program_id, Length: 132, dtype: float64

In [ ]:

all_users = ratings_df[(ratings_df["program_id"].isin(similar_user_programs.index)) & (ratings_df["rating"] >= 4)]

In [ ]:
all_users

,user_id_maped,program_id,rating
6,11374,3.0,4
41,18348,33.0,4
52,730,38.0,4
72,18960,55.0,4
125,29524,87.0,4
...,...,...,...
1048515,26231,328.0,4
1048516,23663,2831.0,4
1048533,29320,1283.0,4
1048549,10318,271.0,4


In [ ]:
all_user_recs = all_users["program_id"].value_counts() / len(all_users["user_id_maped"].unique())

In [ ]:
all_user_recs

709.0     0.622432
328.0     0.610307
603.0     0.606938
581.0     0.595150
272.0     0.573931
            ...   
4087.0    0.019198
530.0     0.018525
32.0      0.017514
5287.0    0.013809
565.0     0.013473
Name: program_id, Length: 132, dtype: float64

In [ ]:
percentages = pd.concat([similar_user_programs, all_user_recs], axis=1)
percentages.columns = ["similar", "all"]

In [ ]:
percentages

,similar,all
2.0,2.850174,0.275514
53.0,1.585366,0.497474
24.0,1.104530,0.236106
709.0,0.933798,0.622432
603.0,0.459930,0.606938
...,...,...
733.0,0.101045,0.130010
1642.0,0.101045,0.073425
3560.0,0.101045,0.116874
3066.0,0.101045,0.113506


In [ ]:
percentages["score"] = percentages["similar"] / percentages["all"]
percentages = percentages.sort_values("score", ascending=False)
percentages.head(10).merge(df, left_index=True, right_on="program_id")


,similar,all,score,Unnamed: 0,user_id_maped,program_name,rating,date_,program_genre,cleaned_title,program_id
1,2.850174,0.275514,10.344948,1,7946,Moana,1,2017-05-21,Animation,moana,2.0
18,2.850174,0.275514,10.344948,18,26546,Moana,1,2017-06-25,Animation,moana,2.0
19,2.850174,0.275514,10.344948,19,20306,Moana,1,2017-06-23,Animation,moana,2.0
48,2.850174,0.275514,10.344948,48,28687,Moana,2,2017-05-30,Animation,moana,2.0
191,2.850174,0.275514,10.344948,191,16739,Moana,1,2017-05-27,Animation,moana,2.0
...,...,...,...,...,...,...,...,...,...,...,...
1031550,0.111498,0.020209,5.517305,1031550,262,The pongeBob quarePants Movie,2,2017-11-02,Animation,the pongebob quarepants movie,121.0
1031847,0.111498,0.020209,5.517305,1031847,31203,The pongeBob quarePants Movie,2,2017-10-20,Animation,the pongebob quarepants movie,121.0
1033551,0.111498,0.020209,5.517305,1033551,22833,The pongeBob quarePants Movie,3,2017-06-04,Animation,the pongebob quarepants movie,121.0
1037893,0.111498,0.020209,5.517305,1037893,20720,The pongeBob quarePants Movie,3,2018-02-13,Animation,the pongebob quarepants movie,121.0


In [ ]:
# def find_similar_movies(movie_id):
#     similar_users = ratings_df[(ratings_df["program_id"] == movie_id) & (ratings_df["rating"] >= 4)]["user_id_maped"].unique()
#     similar_user_recs = ratings_df[(ratings_df["user_id_maped"].isin(similar_users)) & (ratings_df["rating"] >= 4)]["program_id"]
#     similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

#     similar_user_recs = similar_user_recs[similar_user_recs > .10]
#     all_users = ratings_df[(ratings_df["program_id"].isin(similar_user_recs.index)) & (ratings_df["rating"] >= 4)]
#     all_user_recs = all_users["program_id"].value_counts() / len(all_users["user_id_maped"].unique())
#     rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
#     rec_percentages.columns = ["similar", "all"]

#     rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
#     rec_percentages = rec_percentages.sort_values("score", ascending=False)
#     rec_percentages.head(10).merge(df, left_index=True, right_on="program_id")[["score", "program_name", "program_genre"]]
#     return rec_percentages.head(10).merge(df, left_index=True, right_on="program_id")[["score", "program_name", "program_genre"]]

def find_similar_movies(movie_id):
    similar_users = ratings_df[(ratings_df["program_id"] == movie_id) & (ratings_df["rating"] >= 4)]["user_id_maped"].unique()
    similar_user_recs = ratings_df[(ratings_df["user_id_maped"].isin(similar_users)) & (ratings_df["rating"] >= 4)]["program_id"]
    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)

    similar_user_recs = similar_user_recs[similar_user_recs > .10]
    all_users = ratings_df[(ratings_df["program_id"].isin(similar_user_recs.index)) & (ratings_df["rating"] >= 4)]
    all_user_recs = all_users["program_id"].value_counts() / len(all_users["user_id_maped"].unique())
    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]
    rec_percentages = rec_percentages.sort_values("score", ascending=False)

    # Merge with df to retrieve program information
    merged_df = rec_percentages.merge(df, left_index=True, right_on="program_id")

    # Get unique program names with their information
    unique_programs = merged_df.drop_duplicates(subset=["program_id"])[["score", "program_name", "program_genre"]]

    return unique_programs

In [ ]:
def search(title):
    title = clean_title(title)
    query_vec = vectorizer.transform([title])
    similarity = cosine_similarity(query_vec, tfidf).flatten()

    indices = np.argpartition(similarity, -20)[-20:]
    results = df.iloc[indices].iloc[::-1]

    return results


In [ ]:

movie_name_input = widgets.Text(
    disabled=False
)
recommendation_list = widgets.Output()

def on_type(data):
    with recommendation_list:
        recommendation_list.clear_output()
        title = data["new"]
        if len(title) >= 1:
            results = search(title)
            movie_id = results.iloc[0]["program_id"]
            display(find_similar_movies(movie_id))

movie_name_input.observe(on_type, names='value')

display(movie_name_input, recommendation_list)


Text(value='')

Output()